In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

# Helper Functions

In [2]:
def show_image(image,title="image",cmap_type="gray"):
    plt.imshow(image,cmap_type)
    plt.title(title)
    plt.axis("off")
    
def gaussian_blur(img,kernel_size):
    return cv2.GaussianBlur(img,(kernel_size,kernel_size),#5x5 I guess
                  0)
def pipeline1(img, s_thresh=(100, 255), sx_thresh=(15, 255)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    h_channel = hls[:,:,0]
    v_channel= hsv[:,:,2]
    return l_channel